In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from datetime import datetime, timedelta, timezone

import sys
sys.path.append('../')
from utils import data
from models.lenet import OctLenet as Lenet
from utils.visualization import *

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
bn = False
# const
taskName = 'mnist'
modelName = 'octLenet{}'.format('_bn' if bn else '')
JST = timezone(timedelta(hours=+9), 'JST')
now = datetime.now(JST)
nowStr = now.strftime("%Y%m%d_%H%M%S")
showImgCount = 10

imgHeight = 28
imgWidth = 28
imgChannel = 1
imgShape = [-1, imgHeight, imgWidth, imgChannel]
labelSize = 10

# hyper parameter
bs = 64
lr = 0.0001
ep = 50
alpha = 0.25

logDir = '../logs/{}/{}/{}/'.format(taskName, modelName, nowStr)
mnist = input_data.read_data_sets("data/", one_hot=True)
checkPointDir = '{}images.ckpt'.format(logDir)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# main
with tf.Graph().as_default():
    trainIteration = mnist.train.num_examples // bs
    testIteration = mnist.validation.num_examples // bs
    x = tf.placeholder("float", [None, imgHeight * imgWidth * imgChannel], name='x')
    y = tf.placeholder("float", [None, labelSize], name='label')
    isTrain = tf.placeholder(tf.bool, name='is_train')
    
    network = Lenet(alpha, x, inputReshapeTo=imgShape, labelSize=labelSize, batchNorm=bn, visualization=True)
    _ = network.inference(isTrain)
    cost = network.loss(y)
    optimizer = tf.train.AdamOptimizer(lr)
    accuracy = network.accuracy(y)
    trainOp = network.training(cost, optimizer)
    drawHist = tf.summary.merge_all('histogram')

    # tensor board
    # TODO: draw hist
#    allVariables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    
    saver = tf.train.Saver()
    config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    sess = tf.Session(config=config)
    summaryWriter = tf.summary.FileWriter(logDir, graph=sess.graph)

    sess.run(tf.global_variables_initializer())

    # Training cycle
    for e in range(ep):
        trainAvgAcc = trainAvgLoss = testAvgAcc = testAvgLoss = 0.
        # train
        #sess.run(trainIteratorInitOp)
        for i in range(trainIteration):
            batchX, batchY = mnist.train.next_batch(bs)
            _, l, a, histogram = sess.run([trainOp, cost, accuracy, drawHist], feed_dict={x: batchX, y: batchY, isTrain: True})
            trainAvgAcc += a / trainIteration
            trainAvgLoss += l / trainIteration

        # validation
        for i in range(testIteration):
            testBatchX, testBatchY = mnist.validation.next_batch(bs)
            l, a = sess.run([cost, accuracy], feed_dict={x: testBatchX, y: testBatchY, isTrain: False})
            testAvgAcc += a / testIteration
            testAvgLoss += l / testIteration

        # tensorBoard
        summaryWriter.add_summary(drawSclar('validation', {'acc': testAvgAcc, 'loss': testAvgLoss}), global_step=e)
        summaryWriter.add_summary(drawSclar('train', {'acc': trainAvgAcc, 'loss': trainAvgLoss}), global_step=e)
        summaryWriter.add_summary(histogram, global_step=e)
        saver.save(sess, checkPointDir, global_step=e)
        print('Epoch: {} | Train Accuracy: {:.4f} | Train Loss: {:.4f} | Test Accuracy: {:.4f} | Test Loss: {:.4f}'.format(e, trainAvgAcc, trainAvgLoss, testAvgAcc, testAvgLoss))
    summaryWriter.close()
    sess.close()

Epoch: 0 | Train Accuracy: 0.2260 | Train Loss: 2.2601 | Test Accuracy: 0.3626 | Test Loss: 2.1539
Epoch: 1 | Train Accuracy: 0.5662 | Train Loss: 1.9996 | Test Accuracy: 0.6771 | Test Loss: 1.8570
Epoch: 2 | Train Accuracy: 0.7386 | Train Loss: 1.7958 | Test Accuracy: 0.7756 | Test Loss: 1.7412
Epoch: 3 | Train Accuracy: 0.7782 | Train Loss: 1.7272 | Test Accuracy: 0.7957 | Test Loss: 1.7007
Epoch: 4 | Train Accuracy: 0.7921 | Train Loss: 1.6975 | Test Accuracy: 0.8027 | Test Loss: 1.6793
Epoch: 5 | Train Accuracy: 0.8013 | Train Loss: 1.6811 | Test Accuracy: 0.8101 | Test Loss: 1.6656
Epoch: 6 | Train Accuracy: 0.8069 | Train Loss: 1.6695 | Test Accuracy: 0.8175 | Test Loss: 1.6580
Epoch: 7 | Train Accuracy: 0.8131 | Train Loss: 1.6608 | Test Accuracy: 0.8243 | Test Loss: 1.6484
Epoch: 8 | Train Accuracy: 0.8171 | Train Loss: 1.6542 | Test Accuracy: 0.8257 | Test Loss: 1.6436
Epoch: 9 | Train Accuracy: 0.8210 | Train Loss: 1.6489 | Test Accuracy: 0.8303 | Test Loss: 1.6378
Epoch: 10 